## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [117]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
# week_ending = format_date_with_ordinal(today)
# week_starting = format_date_with_ordinal(one_week_ago)

week_ending = "Nov. 24th"
week_starting = "Nov. 17th"

print(week_starting, week_ending)

Nov. 17th Nov. 24th


In [66]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [67]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [68]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [69]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [70]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,November-18-2024,Condo/Co-op,9431 Boca Cove Cir #1001,Boca Raton,FL,33428.0,256000.0,2.0,2.0,Boca Cove Condo,900.0,NaN,1990.0,NaN,284.0,456.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/9431-Boca...,Beaches MLS,RX-10999328,N,Y,26.328202,-80.193392


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [71]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [72]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
54,PAST SALE,2024-11-25,Condo/Co-op,17 Brittany A,Delray Beach,FL,33446.0,102990.0,2.0,2.0,Kings Point Brittany Condos,880.0,NaN,1973.0,NaN,117.0,660.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/17-Brit...,Beaches MLS,RX-11003612,N,Y,26.443797,-80.153957
67,PAST SALE,2024-11-25,Condo/Co-op,750 NE Spanish River Blvd #403,Boca Raton,FL,33431.0,345000.0,2.0,2.0,Boca House,1056.0,NaN,1979.0,NaN,327.0,816.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/750-NE-Sp...,Beaches MLS,F10444661,N,Y,26.385817,-80.073267
33,PAST SALE,2024-11-25,Condo/Co-op,4736 Chancellor 11 Dr #11,Jupiter,FL,33458.0,260000.0,1.0,1.0,Village AT Abacoa Condo,596.0,90.0,2001.0,NaN,436.0,362.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/4736-Chancel...,Beaches MLS,RX-11019781,N,Y,26.890304,-80.113502
53,PAST SALE,2024-11-25,Condo/Co-op,2024 Guildford B,Boca Raton,FL,33434.0,150000.0,2.0,1.5,Century Village,861.0,NaN,1980.0,NaN,174.0,455.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2024-Guil...,Beaches MLS,RX-11000245,N,Y,26.386803,-80.175392
19,PAST SALE,2024-11-25,Condo/Co-op,170 Horizons E #104,Boynton Beach,FL,33435.0,175900.0,2.0,1.5,Sterling Village 170 Condo,815.0,NaN,1967.0,NaN,216.0,662.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/170-Ho...,Beaches MLS,RX-11028264,N,Y,26.525927,-80.054715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,PAST SALE,2024-11-18,Condo/Co-op,103 Hastings G Unit G,West Palm Beach,FL,33417.0,95000.0,1.0,1.0,Hastings Condo,540.0,NaN,1972.0,NaN,176.0,454.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/103-...,Beaches MLS,RX-11024354,N,Y,26.722612,-80.125769
28,PAST SALE,2024-11-18,Condo/Co-op,330 Crestwood Cir #205,Royal Palm Beach,FL,33411.0,282000.0,3.0,2.0,Kensington Of Royal Palm Beach Condo,1314.0,NaN,2004.0,NaN,215.0,384.0,Sold,NaN,NaN,https://www.redfin.com/FL/Royal-Palm-Beach/330...,Beaches MLS,RX-11013689,N,Y,26.687843,-80.245740
27,PAST SALE,2024-11-18,Condo/Co-op,5951 Wellesley Park Dr #608,Boca Raton,FL,33433.0,280000.0,2.0,2.0,Wellesley Park Condo,1180.0,NaN,1982.0,NaN,237.0,640.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/5951-Well...,Beaches MLS,F10459771,N,Y,26.337388,-80.134641
24,PAST SALE,2024-11-18,Condo/Co-op,336 Golfview Rd #1002,North Palm Beach,FL,33408.0,460000.0,2.0,2.0,Gemini,1410.0,NaN,1975.0,NaN,326.0,1200.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/336...,Beaches MLS,RX-11018623,N,Y,26.834119,-80.061751


In [73]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [74]:
df_top_ten = df.head(10)

## Collect Agent Information

In [75]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [76]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [77]:
response_df = pd.DataFrame(response_list)

In [78]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [79]:
df_top_ten = merged_df

## Current Week's Values

In [80]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

82


In [81]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$386,830


In [82]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$288


In [83]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$31,720,098


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [22]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach County
Input Previous Week Condo Sales Total (number units sold):
62
Input Previous Week Condo Average Sales Price:
809035
Input Previous Week Condo Average PSF:
397
Input Previous Week Condo Sales total (ex: 198_000_000)
50_200_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [84]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [85]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [88]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [87]:
pd.set_option('display.max_columns',None)

In [89]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-21,Condo/Co-op,701 S Olive Ave #1901,West Palm Beach,FL,33401.0,3000000.0,3.0,3.5,Two City Plaza Condo,3000.0,NaN,2008.0,NaN,1000.0,2812.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-11016626,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Frank Owen,Illustrated Properties (Abacoa),Rhonda Shoumate,Illustrated Properties,Colleen Foley,Luxe Life Realty LLC,None,None,1,orange


In [90]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [91]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-21,Condo/Co-op,701 S Olive Ave #1901,West Palm Beach,FL,33401.0,3000000.0,3.0,3.5,Two City Plaza Condo,3000.0,NaN,2008.0,NaN,1000.0,2812.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-11016626,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Frank Owen,Illustrated Properties (Abacoa),Rhonda Shoumate,Illustrated Properties,Colleen Foley,Luxe Life Realty LLC,None,None,1,orange
1,PAST SALE,2024-11-18,Condo/Co-op,4600 N Ocean Dr Ph 1903,Singer Island,FL,33404.0,2450000.0,3.0,3.0,BEACH FRONT,2720.0,NaN,2007.0,NaN,901.0,2671.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4600-N...,MARMLS,A11650320,N,Y,26.797586,-80.032558,https://www.redfin.com/FL/Riviera-Beach/4600-N...,Kevin Dickenson,Berkshire Hathaway HomeSvs FL,None,None,Kevin Dickenson,Berkshire Hathaway HomeSvs FL,None,None,2,blue
2,PAST SALE,2024-11-18,Condo/Co-op,550 SE Mizner Blvd Unit B103,Boca Raton,FL,33432.0,2100000.0,3.0,3.0,Townsend Place,2473.0,NaN,2003.0,NaN,849.0,2958.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-SE-Mi...,Beaches MLS,RX-11038162,N,Y,26.345366,-80.084570,https://www.redfin.com/FL/Boca-Raton/550-SE-Mi...,Jean-Luc Andriot,Keller Williams Realty Services,None,None,Jean-Luc Andriot,Keller Williams Realty Services,None,None,3,blue
3,PAST SALE,2024-11-20,Condo/Co-op,170 Chilean Ave Unit 6b,Palm Beach,FL,33480.0,1600000.0,2.0,2.0,Everglades Plaza Condo,1201.0,NaN,1976.0,NaN,1332.0,1190.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-Chile...,Beaches MLS,RX-10987424,N,Y,26.702209,-80.035947,https://www.redfin.com/FL/Palm-Beach/170-Chile...,Jacob Kodner,Provident Realty of South Florida Inc.,None,None,Kristina McPherson,Compass Florida LLC (PB),None,None,4,blue
4,PAST SALE,2024-11-22,Condo/Co-op,3912 S Ocean Blvd #1415,Highland Beach,FL,33487.0,1200000.0,2.0,2.0,Regency Highland Club Condo,1521.0,359575.0,1976.0,NaN,789.0,1171.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-10978089,N,Y,26.405325,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Scott Gerow,Compass Florida LLC,Rachelle Hirt-Beresh,Compass Florida LLC,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,5,blue
5,PAST SALE,2024-11-22,Condo/Co-op,16647 N Traders Xing N #225,Jupiter,FL,33477.0,760000.0,3.0,2.0,Traders Crossing AT Jonathans Landing,1796.0,NaN,1987.0,NaN,423.0,814.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/16647-Trader...,Beaches MLS,RX-11019638,N,Y,26.922480,-80.091692,https://www.redfin.com/FL/Jupiter/16647-Trader...,Mary Jane McElwaney,"NV Realty Group, LLC",None,None,Carol Kennedy,Jonathan's Resales & Rentals,None,None,6,blue
6,PAST SALE,2024-11-21,Condo/Co-op,9135 Passiflora Way #102,Boca Raton,FL,33428.0,745000.0,2.0,2.0,Boca Flores Condo,1916.0,NaN,2021.0,NaN,389.0,778.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/9135-Pass...,Beaches MLS,RX-11020745,N,Y,26.354293,-80.189014,https://www.redfin.com/FL/Boca-Raton/9135-Pass...,Rochelle LeCavalier,Douglas Elliman,None,None,Diana Fairbanks,Balistreri Real Estate Inc,None,None,7,blue
7,PAST SALE,2024-11-19,Condo/Co-op,1910 S Ocean Blvd #318,Delray Beach,FL,33483.0,600000.0,2.0,2.0,Penthouse Delray Condo,1000.0,150.0,1968.0,NaN,600.0,650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1910-S-...,Beaches MLS,RX-11030445,N,Y,26.437465,-80.064245,https://www.redfin.com/FL/Delray-Beach/1910-S-...,Stephanie Muzzillo,Serhant,None,None,Jennifer Gallagher,Serhant,None,None,8,blue
8,PAST SALE,2024-

In [118]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [93]:
m.save('index.html')

## Data snagger

In [94]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [95]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-21,Condo/Co-op,701 S Olive Ave #1901,West Palm Beach,FL,33401.0,3000000.0,3.0,3.5,Two City Plaza Condo,3000.0,NaN,2008.0,NaN,1000.0,2812.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-11016626,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Frank Owen,Illustrated Properties (Abacoa),Rhonda Shoumate,Illustrated Properties,Colleen Foley,Luxe Life Realty LLC,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [96]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_11252024


# CREATE TEMPLATE 

In [97]:
muni_set = set(df_top_ten['CITY'])

In [98]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [99]:
df_top_ten.reset_index(inplace=True,drop=True)

In [100]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [101]:
top_sale

'Two City Plaza Condo at 701 S Olive Ave #1901 in West Palm Beach'

In [102]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [103]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [104]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-21,Condo/Co-op,701 S Olive Ave #1901,West Palm Beach,FL,33401.0,3000000.0,3.0,3.5,Two City Plaza Condo,3000.0,NaN,2008.0,NaN,1000.0,2812.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-11016626,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Frank Owen,Illustrated Properties (Abacoa),Rhonda Shoumate,Illustrated Properties,Colleen Foley,Luxe Life Realty LLC,None,None,1,orange,Two City Plaza Condo at 701 S Olive Ave #1901 ...
1,PAST SALE,2024-11-18,Condo/Co-op,4600 N Ocean Dr Ph 1903,Singer Island,FL,33404.0,2450000.0,3.0,3.0,BEACH FRONT,2720.0,NaN,2007.0,NaN,901.0,2671.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4600-N...,MARMLS,A11650320,N,Y,26.797586,-80.032558,https://www.redfin.com/FL/Riviera-Beach/4600-N...,Kevin Dickenson,Berkshire Hathaway HomeSvs FL,None,None,Kevin Dickenson,Berkshire Hathaway HomeSvs FL,None,None,2,blue,BEACH FRONT at 4600 N Ocean Dr Ph 1903 in Sing...
2,PAST SALE,2024-11-18,Condo/Co-op,550 SE Mizner Blvd Unit B103,Boca Raton,FL,33432.0,2100000.0,3.0,3.0,Townsend Place,2473.0,NaN,2003.0,NaN,849.0,2958.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-SE-Mi...,Beaches MLS,RX-11038162,N,Y,26.345366,-80.084570,https://www.redfin.com/FL/Boca-Raton/550-SE-Mi...,Jean-Luc Andriot,Keller Williams Realty Services,None,None,Jean-Luc Andriot,Keller Williams Realty Services,None,None,3,blue,Townsend Place at 550 SE Mizner Blvd Unit B103...
3,PAST SALE,2024-11-20,Condo/Co-op,170 Chilean Ave Unit 6b,Palm Beach,FL,33480.0,1600000.0,2.0,2.0,Everglades Plaza Condo,1201.0,NaN,1976.0,NaN,1332.0,1190.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-Chile...,Beaches MLS,RX-10987424,N,Y,26.702209,-80.035947,https://www.redfin.com/FL/Palm-Beach/170-Chile...,Jacob Kodner,Provident Realty of South Florida Inc.,None,None,Kristina McPherson,Compass Florida LLC (PB),None,None,4,blue,Everglades Plaza Condo at 170 Chilean Ave Unit...
4,PAST SALE,2024-11-22,Condo/Co-op,3912 S Ocean Blvd #1415,Highland Beach,FL,33487.0,1200000.0,2.0,2.0,Regency Highland Club Condo,1521.0,359575.0,1976.0,NaN,789.0,1171.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-10978089,N,Y,26.405325,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Scott Gerow,Compass Florida LLC,Rachelle Hirt-Beresh,Compass Florida LLC,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,5,blue,Regency Highland Club Condo at 3912 S Ocean Bl...
5,PAST SALE,2024-11-22,Condo/Co-op,16647 N Traders Xing N #225,Jupiter,FL,33477.0,760000.0,3.0,2.0,Traders Crossing AT Jonathans Landing,1796.0,NaN,1987.0,NaN,423.0,814.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/16647-Trader...,Beaches MLS,RX-11019638,N,Y,26.922480,-80.091692,https://www.redfin.com/FL/Jupiter/16647-Trader...,Mary Jane McElwaney,"NV Realty Group, LLC",None,None,Carol Kennedy,Jonathan's Resales & Rentals,None,None,6,blue,Traders Crossing AT Jonathans Landing at 16647...
6,PAST SALE,2024-11-21,Condo/Co-op,9135 Passiflora Way #102,Boca Raton,FL,33428.0,745000.0,2.0,2.0,Boca Flores Condo,1916.0,NaN,2021.0,NaN,389.0,778.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/9135-Pass...,Beaches MLS,RX-11020745,N,Y,26.354293,-80.189014,https://www.redfin.com/FL/Boca-Raton/9135-Pass...,Rochelle LeCavalier,Douglas Elliman,None,None,Diana Fairbanks,Balistreri Real Estate Inc,None,None,7,blue,Boca Flores Condo at 9135 Passiflora Way #102 ...
7,PAST SALE,2024-11-19,Condo/Co-op,1910 S Ocean Blvd #318,

In [105]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Two City Plaza Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $555,308M to $3,000,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Palm Beach, Highland Beach, Boca Raton, Delray Beach, Singer Island, Jupiter, West Palm Beach, Riviera Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 82 condo sales totaling $31,720,098 million from Nov. 17th to Nov. 25th. The previous week, brokers closed 62 condo sales totaling $50,200,000.

Last week’s units sold for an average of

In [106]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [107]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [108]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [109]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-21,Condo/Co-op,701 S Olive Ave #1901,West Palm Beach,FL,33401.0,3000000.0,3.0,3.5,Two City Plaza Condo,3000.0,NaN,2008.0,NaN,1000.0,2812.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-11016626,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Frank Owen,Illustrated Properties (Abacoa),Rhonda Shoumate,Illustrated Properties,Colleen Foley,Luxe Life Realty LLC,None,None,1,orange,Two City Plaza Condo at 701 S Olive Ave #1901 ...
1,PAST SALE,2024-11-18,Condo/Co-op,4600 N Ocean Dr Ph 1903,Singer Island,FL,33404.0,2450000.0,3.0,3.0,BEACH FRONT,2720.0,NaN,2007.0,NaN,901.0,2671.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4600-N...,MARMLS,A11650320,N,Y,26.797586,-80.032558,https://www.redfin.com/FL/Riviera-Beach/4600-N...,Kevin Dickenson,Berkshire Hathaway HomeSvs FL,None,None,Kevin Dickenson,Berkshire Hathaway HomeSvs FL,None,None,2,blue,BEACH FRONT at 4600 N Ocean Dr Ph 1903 in Sing...
2,PAST SALE,2024-11-18,Condo/Co-op,550 SE Mizner Blvd Unit B103,Boca Raton,FL,33432.0,2100000.0,3.0,3.0,Townsend Place,2473.0,NaN,2003.0,NaN,849.0,2958.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-SE-Mi...,Beaches MLS,RX-11038162,N,Y,26.345366,-80.084570,https://www.redfin.com/FL/Boca-Raton/550-SE-Mi...,Jean-Luc Andriot,Keller Williams Realty Services,None,None,Jean-Luc Andriot,Keller Williams Realty Services,None,None,3,blue,Townsend Place at 550 SE Mizner Blvd Unit B103...
3,PAST SALE,2024-11-20,Condo/Co-op,170 Chilean Ave Unit 6b,Palm Beach,FL,33480.0,1600000.0,2.0,2.0,Everglades Plaza Condo,1201.0,NaN,1976.0,NaN,1332.0,1190.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-Chile...,Beaches MLS,RX-10987424,N,Y,26.702209,-80.035947,https://www.redfin.com/FL/Palm-Beach/170-Chile...,Jacob Kodner,Provident Realty of South Florida Inc.,None,None,Kristina McPherson,Compass Florida LLC (PB),None,None,4,blue,Everglades Plaza Condo at 170 Chilean Ave Unit...
4,PAST SALE,2024-11-22,Condo/Co-op,3912 S Ocean Blvd #1415,Highland Beach,FL,33487.0,1200000.0,2.0,2.0,Regency Highland Club Condo,1521.0,359575.0,1976.0,NaN,789.0,1171.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-10978089,N,Y,26.405325,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Scott Gerow,Compass Florida LLC,Rachelle Hirt-Beresh,Compass Florida LLC,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,5,blue,Regency Highland Club Condo at 3912 S Ocean Bl...
5,PAST SALE,2024-11-22,Condo/Co-op,16647 N Traders Xing N #225,Jupiter,FL,33477.0,760000.0,3.0,2.0,Traders Crossing AT Jonathans Landing,1796.0,NaN,1987.0,NaN,423.0,814.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/16647-Trader...,Beaches MLS,RX-11019638,N,Y,26.922480,-80.091692,https://www.redfin.com/FL/Jupiter/16647-Trader...,Mary Jane McElwaney,"NV Realty Group, LLC",None,None,Carol Kennedy,Jonathan's Resales & Rentals,None,None,6,blue,Traders Crossing AT Jonathans Landing at 16647...
6,PAST SALE,2024-11-21,Condo/Co-op,9135 Passiflora Way #102,Boca Raton,FL,33428.0,745000.0,2.0,2.0,Boca Flores Condo,1916.0,NaN,2021.0,NaN,389.0,778.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/9135-Pass...,Beaches MLS,RX-11020745,N,Y,26.354293,-80.189014,https://www.redfin.com/FL/Boca-Raton/9135-Pass...,Rochelle LeCavalier,Douglas Elliman,None,None,Diana Fairbanks,Balistreri Real Estate Inc,None,None,7,blue,Boca Flores Condo at 9135 Passiflora Way #102 ...
7,PAST SALE,2024-11-19,Condo/Co-op,1910 S Ocean Blvd #318,

In [110]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: Two City Plaza Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $555,308M to $3,000,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Palm Beach, Highland Beach, Boca Raton, Delray Beach, Singer Island, Jupiter, West Palm Beach, Riviera Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 82 condo sales totaling $31,720,098 million from Nov. 17th to Nov. 25th. The previous week, brokers closed 62 condo sales totaling $50,200,000.

Last week’s units sold for an average of

In [111]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/West-Palm-Beach/701-S-Olive-Ave-33401/unit-1901/home/42661544


In [112]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Riviera-Beach/4600-N-Ocean-Dr-33404/unit-1903/home/42658070


In [113]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Highland-Beach/3310-S-Ocean-Blvd-33487/unit-232D/home/42385554


In [114]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Riviera-Beach/5480-N-Ocean-Dr-33404/unit-A6D/home/42456373


In [115]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/170-Chilean-Ave-33480/unit-6B/home/42442340


In [116]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Boca-Raton/9135-Passiflora-Way-33428/unit-102/home/184423704


In [58]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-21,Condo/Co-op,701 S Olive Ave #1901,West Palm Beach,FL,33401.0,3000000.0,3.0,3.5,Two City Plaza Condo,3000.0,NaN,2008.0,NaN,1000.0,2812.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-11016626,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Frank Owen,Illustrated Properties (Abacoa),Rhonda Shoumate,Illustrated Properties,Colleen Foley,Luxe Life Realty LLC,None,None,1,orange,Two City Plaza Condo at 701 S Olive Ave #1901 ...
1,PAST SALE,2024-11-19,Condo/Co-op,332 Spyglass Way,Jupiter,FL,33477.0,2500000.0,2.0,2.0,Admirals Cove,2275.0,NaN,1988.0,NaN,1099.0,2277.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/332-Spyglass...,Beaches MLS,RX-11021531,N,Y,26.911561,-80.088739,https://www.redfin.com/FL/Jupiter/332-Spyglass...,Thomas Frankel,Admirals Cove Realty Co Inc,Allan Meyerson,Admirals Cove Realty Co Inc,Thomas Frankel,Admirals Cove Realty Co Inc,None,None,2,blue,Admirals Cove at 332 Spyglass Way in Jupiter
2,PAST SALE,2024-11-18,Condo/Co-op,4600 N Ocean Dr Ph 1903,Singer Island,FL,33404.0,2450000.0,3.0,3.0,BEACH FRONT,2720.0,NaN,2007.0,NaN,901.0,2671.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4600-N...,MARMLS,A11650320,N,Y,26.797586,-80.032558,https://www.redfin.com/FL/Riviera-Beach/4600-N...,Kevin Dickenson,Berkshire Hathaway HomeSvs FL,None,None,Kevin Dickenson,Berkshire Hathaway HomeSvs FL,None,None,3,blue,BEACH FRONT at 4600 N Ocean Dr Ph 1903 in Sing...
3,PAST SALE,2024-11-18,Condo/Co-op,550 SE Mizner Blvd Unit B103,Boca Raton,FL,33432.0,2100000.0,3.0,3.0,Townsend Place,2473.0,NaN,2003.0,NaN,849.0,2958.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-SE-Mi...,Beaches MLS,RX-11038162,N,Y,26.345366,-80.084570,https://www.redfin.com/FL/Boca-Raton/550-SE-Mi...,Jean-Luc Andriot,Keller Williams Realty Services,None,None,Jean-Luc Andriot,Keller Williams Realty Services,None,None,4,blue,Townsend Place at 550 SE Mizner Blvd Unit B103...
4,PAST SALE,2024-11-20,Condo/Co-op,170 Chilean Ave Unit 6b,Palm Beach,FL,33480.0,1600000.0,2.0,2.0,Everglades Plaza Condo,1201.0,NaN,1976.0,NaN,1332.0,1190.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-Chile...,Beaches MLS,RX-10987424,N,Y,26.702209,-80.035947,https://www.redfin.com/FL/Palm-Beach/170-Chile...,Jacob Kodner,Provident Realty of South Florida Inc.,None,None,Kristina McPherson,Compass Florida LLC (PB),None,None,5,blue,Everglades Plaza Condo at 170 Chilean Ave Unit...
5,PAST SALE,2024-11-22,Condo/Co-op,3912 S Ocean Blvd #1415,Highland Beach,FL,33487.0,1200000.0,2.0,2.0,Regency Highland Club Condo,1521.0,359575.0,1976.0,NaN,789.0,1171.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-10978089,N,Y,26.405325,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Scott Gerow,Compass Florida LLC,Rachelle Hirt-Beresh,Compass Florida LLC,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,6,blue,Regency Highland Club Condo at 3912 S Ocean Bl...
6,PAST SALE,2024-11-22,Condo/Co-op,16647 N Traders Xing N #225,Jupiter,FL,33477.0,760000.0,3.0,2.0,Traders Crossing AT Jonathans Landing,1796.0,NaN,1987.0,NaN,423.0,814.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/16647-Trader...,Beaches MLS,RX-11019638,N,Y,26.922480,-80.091692,https://www.redfin.com/FL/Jupiter/16647-Trader...,Mary Jane McElwaney,"NV Realty Group, LLC",None,None,Carol Kennedy,Jonathan's Resales & Rentals,None,None,7,blue,Traders Crossing AT Jonathans Landing at 16647...
7,PAST SALE,2024-11-21,Condo/Co-op,

## Time on Market Calculator

In [65]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 29) ## List (Earlier) date
date2 = datetime(2024, 11, 21) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

84


## Clean CSV for Datawrapper Chart

In [119]:
chart_df = df_top_ten

In [120]:
chart_df = chart_df.fillna(" ")

In [121]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [122]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [123]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [124]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [125]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [126]:
csv_date_string = today.strftime("%m_%d_%Y")

In [127]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [128]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-11-21,701 S Olive Ave #1901 in West Palm Beach,"$3,000,000",3,3.5,"3,000",2008,"$1,000",Beaches MLS,RX-11016626,Frank Owen Illustrated Properties (Abacoa) Rho...,Colleen Foley Luxe Life Realty LLC
1,2024-11-18,4600 N Ocean Dr Ph 1903 in Singer Island,"$2,450,000",3,3.0,"2,720",2007,$901,MARMLS,A11650320,Kevin Dickenson Berkshire Hathaway HomeSvs FL,Kevin Dickenson Berkshire Hathaway HomeSvs FL
2,2024-11-18,550 SE Mizner Blvd Unit B103 in Boca Raton,"$2,100,000",3,3.0,"2,473",2003,$849,Beaches MLS,RX-11038162,Jean-Luc Andriot Keller Williams Realty Servic...,Jean-Luc Andriot Keller Williams Realty Servic...
3,2024-11-20,170 Chilean Ave Unit 6b in Palm Beach,"$1,600,000",2,2.0,"1,201",1976,"$1,332",Beaches MLS,RX-10987424,Jacob Kodner Provident Realty of South Florida...,Kristina McPherson Compass Florida LLC (PB)
4,2024-11-22,3912 S Ocean Blvd #1415 in Highland Beach,"$1,200,000",2,2.0,"1,521",1976,$789,Beaches MLS,RX-10978089,Scott Gerow Compass Florida LLC Rachelle Hirt-...,Matthew Bachrad William Raveis Real Estate Dan...
5,2024-11-22,16647 N Traders Xing N #225 in Jupiter,"$760,000",3,2.0,"1,796",1987,$423,Beaches MLS,RX-11019638,"Mary Jane McElwaney NV Realty Group, LLC",Carol Kennedy Jonathan's Resales & Rentals
6,2024-11-21,9135 Passiflora Way #102 in Boca Raton,"$745,000",2,2.0,"1,916",2021,$389,Beaches MLS,RX-11020745,Rochelle LeCavalier Douglas Elliman,Diana Fairbanks Balistreri Real Estate Inc
7,2024-11-19,1910 S Ocean Blvd #318 in Delray Beach,"$600,000",2,2.0,"1,000",1968,$600,Beaches MLS,RX-11030445,Stephanie Muzzillo Serhant,Jennifer Gallagher Serhant
8,2024-11-22,5480 N Ocean Dr Unit A6d in Riviera Beach,"$585,000",2,2.0,"1,330",1972,$440,Beaches MLS,RX-11007035,Corinne Misenar William Raveis Real Estate Tar...,Claudia Krinke Douglas Elliman
9,2024-11-21,3310 S Ocean Blvd Unit 232-D in Highland Beach,"$555,308",2,2.0,"1,239",1970,$448,Beaches MLS,RX-11027920,Derek West Coldwell Banker,Scott Porter John P. O'Grady LLC
